# Vežbe 12: Lex, Yacc, LLVM

[Lex](https://en.wikipedia.org/wiki/Lex_(software)) je računarski program koji generiše leksičke analizatore, tj. leksere/skenere. Lex se ubično koristi uz Yacc generator parsera. Struktura Lex datoteke namerno je slična strukturi Yacc datoteke koja je podeljena u tri odeljka, razdvojena linijama koje sadrže samo dva procenta znakova, kao što sledi:

1. **Odeljak definicije** definiše makronaredbe i importuje zaglavlja napisana u C programskom jeziku. Ovde je takođe moguće napisati bilo koji C kod koji će se kopirati u generisanoj izvornoj datoteci.

2. **Odeljak sa pravilima** povezuje regularne izaze sa C semantičkim strukturama. Kada lekser vidi tekst na ulazu koji odgovara datom pravilu, izvršiće pridruženi C kod.

3. **Odeljak C koda** sadrži C semantičke strukture i funkcije koje se kopiraju u generisanu izvornu datoteku. Ovaj odeljak sadrži kod koji se poziva pravilima u prethodnom odeljku.

[Yacc](https://en.wikipedia.org/wiki/Yacc) je generator parsera tipa Look-Ahead-Left-to-Right (LALR). Ulaz u Yacc je gramatika sa pridruženim isečcima C koda. Njegov izlaz je Shift-Reduce (SR) parser napisan u C programskom jeziku koji izvršava C kod povezan sa svakim pravilom čim se to pravilo prepozna. Generisanje parsera podrazumeva prethodno generisanje AST.

[LLVM](https://drive.google.com/file/d/1UE45xzbdkS_l5hPfzFM6dIR05ZqITz7w/view?usp=sharing) je infrastruktura kompajlera sa skupom alatki za njihovu konstrukciju koje se koristi za ravoj Front-End dela kompajlera za bilo koji programski jezik i Back-End dela kompajlera za bilo koju arhitekturu. LLVM je dizajniran oko jezički nezavisne međureprezentacije koja služi kao portabilni assembly kod visokog nivoa koji se može višestruko optimizovati.

Testiranje Lex/Yacc implementacije.

Autor: Slađan Kantar

In [ ]:
path = '/content/drive/My Drive/PP/data'
!cp -R '{path}/ply' .
!cp '{path}/ply_test.py' .
!python3 'ply_test.py'

NAPOMENA: Ćelija ispod se ne može izvršiti kroz IPython.

In [ ]:
tokens = (
    'NAME', 'NUMBER',
    'PLUS', 'MINUS', 'TIMES', 'DIVIDE', 'EQUALS',
    'LPAREN', 'RPAREN'
)

t_PLUS      = r'\+'
t_MINUS     = r'-'
t_TIMES     = r'\*'
t_DIVIDE    = r'/'
t_EQUALS    = r'='
t_LPAREN    = r'\('
t_RPAREN    = r'\)'
t_NAME      = r'[a-zA-Z_][a-zA-Z0-9_]*'
t_ignore    = r' \t'

def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)

lex.lex()

precedence = (
    ('left', 'PLUS', 'MINUS'),
    ('left', 'TIMES', 'DIVIDE'),
    ('right', 'UMINUS')
)

names = {}

def p_statement_assign(p):
    'statement : NAME EQUALS expression'
    names[p[1]] = p[3]

def p_statement_expr(p):
    'statement : expression'
    print(p[1])

def p_expression_binop(p):
    '''expression : expression PLUS expression
                  | expression MINUS expression
                  | expression TIMES expression
                  | expression DIVIDE expression'''
    if p[2] == '+'  : p[0] = p[1] + p[3]
    elif p[2] == '-': p[0] = p[1] - p[3]
    elif p[2] == '*': p[0] = p[1] * p[3]
    elif p[2] == '/': p[0] = p[1] / p[3]

def p_expression_uminus(p):
    'expression : MINUS expression %prec UMINUS'
    p[0] = -p[2]

def p_expression_group(p):
    'expression : LPAREN expression RPAREN'
    p[0] = p[2]

def p_expression_number(p):
    'expression : NUMBER'
    p[0] = p[1]

def p_expression_name(p):
    'expression : NAME'
    try:
        p[0] = names[p[1]]
    except LookupError:
        print("Undefined name '%s'" % p[1])
        p[0] = 0

def p_error(p):
    print("Syntax error at '%s'" % p.value)

yacc()

while True:
    try:
        s = input(">>> ")
    except EOFError:
        break
    parse(s)

Testiranje [llvmlite](https://github.com/numba/llvmlite) biblioteke.

In [ ]:
from llvmlite import ir

In [ ]:
m = ir.Module()

In [ ]:
fnty = ir.FunctionType(ir.IntType(32), [ir.IntType(32)])

print(fnty)

In [ ]:
fn = ir.Function(m, fnty, 'count_number')
fn.args[0].name= 'N'

print(fn)

In [ ]:
builder = ir.IRBuilder(fn.append_basic_block('entry'))

In [ ]:
out = builder.alloca(ir.IntType(32), name='out')
ct = builder.alloca(ir.IntType(32), name='ct')

builder.store(out.type.pointee(0), out)
builder.store(ct.type.pointee(0), ct)

print(fn)

In [ ]:
loophead = fn.append_basic_block('loop.header')
loopbody = fn.append_basic_block('loop.body')
loopend = fn.append_basic_block('loop.end')

builder.branch(loophead)
builder.position_at_end(loophead)

# loop if ct < arg0
arg0 = fn.args[0]
pred = builder.icmp_signed('<', builder.load(ct), arg0)
builder.cbranch(pred, loopbody, loopend)

print(fn)

In [ ]:
builder.position_at_end(loopbody)

# out += ct
builder.store(builder.add(builder.load(out), builder.load(ct)), out)
# ct += 1
builder.store(builder.add(builder.load(ct), ct.type.pointee(1)), ct)
# jump to loophead
builder.branch(loophead)

print(fn)

In [ ]:
builder.position_at_end(loopend)

builder.ret(builder.load(out))

print(fn)

In [ ]:
from llvmlite import binding as llvm

In [ ]:
dot = llvm.get_function_cfg(fn)
llvm.view_dot_graph(dot)